In [90]:
import numpy as np
import pandas as pd

In [91]:
df=pd.read_csv("Spotify Million Song Dataset_exported.csv")
df=df.drop(['link'],axis=1)
df.head()

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [92]:
df_new = pd.DataFrame(columns=['artist', 'song', 'text'])

In [93]:
indent = ["\n", "\r", "\t"]
def remove_indents(text, indent):
    for elements in indent:
        text = text.replace(elements, '')
    return text

df['text'] = df['text'].apply(lambda elements: remove_indents(elements, indent))

In [94]:
df['text'] = df['text'].str.lower()


In [95]:
import re
def remove_words_with_apostrophes(input_string):
    pattern = r'\b\w+\'\w+\b'
    result = re.sub(pattern, '', input_string)
    return result

df['text'] = df['text'].apply(lambda elements: remove_words_with_apostrophes(elements))

In [96]:
import string
def remove_punctuation(input_string):
    translation_table = str.maketrans('', '', string.punctuation)
    result = input_string.translate(translation_table)
    return result

df['text'] = df['text'].apply(lambda elements: remove_punctuation(elements))

In [97]:
len(df['text'][0])

694

In [98]:
df_new = pd.DataFrame(columns=['artist', 'song', 'text'])

# Create an empty list to store dictionaries
'''def split_text(text):
    length = len(text)
    split_length = length // 5
    return [text[i:i+split_length] for i in range(0, length, split_length)]'''

def split_text(text):
    words = text.split()
    num_words = len(words)
    split_length = num_words // 10
    return [' '.join(words[i:i+split_length]) for i in range(0, num_words, split_length)]

# Apply the function to the 'text' column and create a new dataframe
df_new = df.copy()
df_new['text'] = df_new['text'].apply(split_text)

# Explode the list of substrings into separate rows
df_new = df_new.explode('text')

# Reset index
df_new.reset_index(drop=True, inplace=True)

df_new


,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,look at her face a wonderful face and it means...
1,ABBA,Ahe's My Kind Of Girl,look at the way that she smiles when she sees ...
2,ABBA,Ahe's My Kind Of Girl,one fellow be just my kind of girl she makes m...
3,ABBA,Ahe's My Kind Of Girl,could ever believe that she could be mine just...
4,ABBA,Ahe's My Kind Of Girl,her blue and if she ever leaves me what could ...
...,...,...,...
628927,Zwan,Heartsong,the same words to say the same things how can ...
628928,Zwan,Heartsong,i need you just be sure no more no less let lo...
628929,Zwan,Heartsong,to you what you must do how can you say that i...
628930,Zwan,Heartsong,you just be sure no more no less let love conf...


In [99]:
'''from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
lyrics = df_new['text'].apply(lambda x: ' '.join([ps.stem(word) for word in re.findall(r'\b\w+\b', x.lower()) if word not in stop_words]))'''

"from nltk.corpus import stopwords\nfrom nltk.stem import PorterStemmer\n\nstop_words = set(stopwords.words('english'))\nps = PorterStemmer()\nlyrics = df_new['text'].apply(lambda x: ' '.join([ps.stem(word) for word in re.findall(r'\x08\\w+\x08', x.lower()) if word not in stop_words]))"

In [100]:
# Vectorization using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(lyrics)
X = vectorizer.fit_transform(df_new['text'])

In [101]:
df['text'][0]

'look at her face  a wonderful face  and it means something special to me  look at the way that she smiles when she sees me  how lucky can one fellow be     just my kind of girl she makes me feel fine  who could ever believe that she could be mine   just my kind of girl without her  blue  and if she ever leaves me what could i do what could i do    and when we go for a walk in the park  and she holds me and squeezes my hand   go on walking for hours and talking  about all the things that we plan     just my kind of girl she makes me feel fine  who could ever believe that she could be mine   just my kind of girl without her  blue  and if she ever leaves me what could i do what could i do'

In [102]:
# Input snippet
input_snippet = "look at her face  a wonderful face"
#input_snippet = "i can hear how you work practising hard  playing night and day wooh  and it sounds better now yes you improve  everytime you play wooh  but  bad   so sad  and  only smiling  when you play your violin    dumb dumb diddle to be your fiddle  to be so near you and not just hear you  dumb dumb diddle to be your fiddle  i think then maybe  see me baby   be mine  and  be together all the time  wish i was dumb dumb diddle your darling fiddle    but i think you  know that i exist   the quiet kind wooh  "

In [103]:
input_snippet=remove_words_with_apostrophes(input_snippet)
input_snippet=remove_punctuation(input_snippet)

In [104]:
# Preprocess input snippet

#input_snippet = ' '.join([ps.stem(word) for word in re.findall(r'\b\w+\b', input_snippet.lower()) if word not in stop_words])

In [105]:
# Vectorize input snippet
input_snippet_vector = vectorizer.transform([input_snippet])

In [106]:
# Calculate similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(X, input_snippet_vector)

In [107]:
# Identify the unique song
unique_song_index = similarity_scores.argmax()
unique_song = df_new.iloc[unique_song_index]

In [108]:
print(f'Artist: {unique_song["artist"]}')
print(f'Song: {unique_song["song"]}')

Artist: ABBA
Song: Ahe's My Kind Of Girl
